In [1]:
from keras.models import load_model
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
def load_pickle_model(filename):
    with open(filename, 'rb') as file:
        model=pickle.load(file)
    return model

In [3]:
# extract features of given image
def extract_features(model, image):
    image=image.resize((299,299))
    image=np.expand_dims(image, axis=0)
    image=image/255
    feature=model.predict(image)
    return feature

In [4]:
# generate description of given image
def generate_description(model, tokenizer, features, max_length):
    in_text=""
    sequence=np.zeros((1,max_length))
    for i in range(max_length):
        pred=model.predict([[features, sequence]], verbose=0)
        token=np.argmax(pred)
        word=tokenizer.index_word.get(token)
        sequence[0][i]=token

        if(not word):
            break
        in_text+=" "+word
        if(word=="endseq"):
            break

    in_text=in_text.replace("startseq", "")
    in_text=in_text.replace("endseq", "")

    return in_text

In [5]:
xception_model=load_pickle_model("data_models/xception_model.pickle")
main_model=load_model("data_models/image_captioning_model.keras")

In [6]:
tokenizer=load_pickle_model("data_models/tokenizer.pickle")
vocab_size=len(tokenizer.word_index)+1
max_length=186

Generating Caption

In [7]:
def generate_image_caption(image_path):
    try:
        image=Image.open(image_path)
    except:
        print("Invalid Image!")
        return None, None

    print("Extracting Image Features...")
    features=extract_features(xception_model, image)
    print("Generating Image Caption")
    generated_desc=generate_description(main_model, tokenizer, features, max_length)
    return image, generated_desc

In [8]:
def show_image_caption(filename):
    image, generated_desc=generate_image_caption(filename)

    if(image):
        plt.title("Image Caption: "+generated_desc)
        plt.axis('off')
        plt.imshow(image, )

In [ ]:
filename="Flickr8k_Dataset/Flicker8k_images/47870024_73a4481f7d.jpg"
show_image_caption(filename)

In [ ]:
filename="Flickr8k_Dataset/Flicker8k_images/3515904775_f8acc5909e.jpg"
show_image_caption(filename)

In [ ]:
filename="Flickr8k_Dataset/Flicker8k_images/1258913059_07c613f7ff.jpg"
show_image_caption(filename)